In [ ]:
import onnx
import numpy as np
from PIL import Image
from urllib import request
from io import BytesIO
import onnxruntime as ort

## Question 1

In [ ]:
model = onnx.load("hair_classifier_v1.onnx")
graph = model.graph

for node in graph.output:
    print(node.name)

## Question 3

In [ ]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

def download_image(url):
    with request.urlopen(url) as resp:
        buf = resp.read()
    return Image.open(BytesIO(buf))

def prepare_image(img):
    if img.mode != "RGB":
        img = img.convert("RGB")
    img = img.resize((200, 200), Image.NEAREST)
    return img

url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"

img = prepare_image(download_image(url))
img_np = np.array(img).astype("float32") / 255.0

# Normalize
img_np = (img_np - mean) / std

print("first pixel, R channel:", img_np[0,0,0])


## Question 4

In [ ]:
img_chw = np.transpose(img_np, (2,0,1))  # HWC → CHW
img_chw = np.expand_dims(img_chw, axis=0).astype("float32")  # add batch dim

session = ort.InferenceSession("hair_classifier_v1.onnx")

input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

pred = session.run([output_name], {input_name: img_chw})[0]
print(pred)